## LLM 모델 활용하여 사용자 리뷰 데이터 긍정부정 구분하는 예시

### *1. 필수 라이브러리 설치 및 로드(+API KEY 설정)*

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_community
!pip install langchain_core
!pip install langchain_openai
!pip install python-dotenv
!pip install langchain_experimental
!pip install sentence_transformers
!pip install faiss-cpu

In [5]:
# import
import pandas as pd

from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import CSVLoader, DataFrameLoader

In [122]:
# API KEY 저장을 위한 os라이브러리 호출
import os

# open API KEY 저장
os.environ['OPEN_API_KEY'] = '여러분이 발급받은 key'

### *2. 데이터 로드*

In [34]:
df = pd.read_excel(r'/content/4_3_naver_user_크롤링_user_df_0_9_최신_지은.xlsx')
df

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count
0,SunnyDay15,장안면옥 명일동점,냉면,서울특별시 강동구 명일동,정보없음,바로 입장,데이트,부모님,갈비탕 맛있어요,'음식이 맛있어요',5월 30일 목요일,목요일,5
1,SunnyDay15,스트라이프커피,카페,서울특별시 강남구 청담동,예약 없이,바로 입장,일상,혼자,동네 커피 맛집 원두가 고소하면서도 묵직해서 원두 \n사와서 집에서 드립으로 내려마...,"'커피가 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 29일 수요일,수요일,4
2,SunnyDay15,홍영재장수청국장,한정식,서울특별시 강남구 대치동,예약 없이,바로 입장,일상,연인·배우자,갈비찜 정식 먹었는데 부드럽고 맛있어요 건강한 밥상,"'음식이 맛있어요', '재료가 신선해요', '친절해요', '주차하기 편해요'",5월 29일 수요일,수요일,1
3,SunnyDay15,크루아상148 위례점,베이커리,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,아메리카노 2500원 무난한 맛이에요 아몬드 크로아상은 조금 드라이하게 느껴졌어요,"'가성비가 좋아요', '친절해요'",5월 28일 화요일,화요일,1
4,SunnyDay15,미도인 위례,양식,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,큐브덮밥 미도인 가케소바 무난 새우커틀렛은 다진 새우를 크로켓 모양으로 튀겼는데 좀...,"'인테리어가 멋져요', '주차하기 편해요'",5월 28일 화요일,화요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,samestage,LTP 한남,브런치,서울특별시 용산구 한남동,예약 후,바로 입장,데이트,아이,평일인데도 웨이팅 있는 핫플이네요\n시그니처브런치 비주얼도이쁘고 맛도 좋아요 \n사...,"'음식이 맛있어요', '재료가 신선해요', '메뉴 구성이 알차요', '인테리어가 멋...",1월 26일 금요일,금요일,1
1285,samestage,메이베이크,"카페,디저트",서울특별시 성동구 하왕십리동,예약 없이,바로 입장,일상,혼자,케이크 맛있는 상왕십리카페 메이베이크 딸기케이크 홀딱 반했어요 미니케이크라 3인가족...,"'디저트가 맛있어요', '빵이 맛있어요', '아늑해요', '친절해요'",1월 25일 목요일,목요일,1
1286,samestage,김치옥 삼송점,"육류,고기요리",경기도 고양시 덕양구 삼송동,예약 없이,바로 입장,데이트,연인·배우자,김치 전문점이라 그런가 김치찜이 엄지척입니다 제육볶음이랑 세뜨로 먹었어요 고기도 찐...,"'음식이 맛있어요', '반찬이 잘 나와요', '메뉴 구성이 알차요', '친절해요'",1월 24일 수요일,수요일,2
1287,samestage,AVEK CHERI,"카페,디저트",서울특별시 용산구 한남동,예약 없이,바로 입장,일상,혼자,소금빵맛집 잘찾아온거같아요 소프트소금빵 오리지널소금빵 둘다먹었는데 각자의매력이 있어...,"'커피가 맛있어요', '빵이 맛있어요', '인테리어가 멋져요', '뷰가 좋아요', ...",1월 22일 월요일,월요일,1


#### place_cate, place_review, place_tag 전처리

In [46]:
# 제거할 카테고리, 리뷰, 태그
remove_cate = ['동물병원', '아울렛', '종합생활용품', '슈퍼,마트', '사진,스튜디오','미용실','꽃집,꽃배달','패션잡화', '헬스장', '정보없음', '워터파크',
               'GS칼텍스주유소', '편의점', '서점', '안과', '약국', '패션', '캐주얼웨어', '문구,팬시용품', '영화관', '고속도로휴게소', '치과',
               '스포츠용품', '백화점', '푸드트럭', '드럭스토어', '불교용품', '공사,공단', '지하철,전철', '신발',  '인테리어소품', '오락실', '종합패션',
               '헤어악세서리',  '애플리케이션개발', '금융서비스', '기능성화장품',  '은행', '통신', '휴대폰', 'IT서비스', '카셰어링', '인터넷상거래',
               '신용카드', '도시가스', '기업', '도서관', '급식', '대리석제조', '물류센터', '우체국', '출력,복사,제본', '과학기술정보통신부',
               '시민단체', '소프트웨어개발', '지불,결제대행', '내과', '도서,음반,문구', '지방행정기관', '남성정장', '스크린골프장',  '마취통증의학과',
               '한의원', '피부,체형관리', '정신건강의학과', '노래방', '생활협동조합', '민영주차장', '반려동물미용', '타월,수건', 'SK주유소', '병원,의원',
               '왁싱,제모',  'S', 'HD현대오일뱅크주유소', '공영주차장', '장소대여', '면허시험장', '네일아트,네일샵', '메이크업', '캠코더,카메라', '주차장',
               '호텔', '도시,테마공원', '섬', '레저,테마',  '호수,연못,저수지', '산',  '찜질방', '보세의류', '전망대', 'PC방', '하천', '유원지', '방면정보',
               '애견사료', '주유소', '보드카페', '안경원', '화훼재배', '농산물창고', '테마카페', '식물원,수목원', '정형외과']


new_df = df[~df['place_cate'].isin(remove_cate)]  # ~ 연산자는 불리언 시리즈를 반전
new_df = new_df[new_df['place_review']!='정보없음']
new_df = new_df[new_df['place_tag']!='정보없음']
new_df

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count
0,SunnyDay15,장안면옥 명일동점,냉면,서울특별시 강동구 명일동,정보없음,바로 입장,데이트,부모님,갈비탕 맛있어요,'음식이 맛있어요',5월 30일 목요일,목요일,5
1,SunnyDay15,스트라이프커피,카페,서울특별시 강남구 청담동,예약 없이,바로 입장,일상,혼자,동네 커피 맛집 원두가 고소하면서도 묵직해서 원두 \n사와서 집에서 드립으로 내려마...,"'커피가 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 29일 수요일,수요일,4
2,SunnyDay15,홍영재장수청국장,한정식,서울특별시 강남구 대치동,예약 없이,바로 입장,일상,연인·배우자,갈비찜 정식 먹었는데 부드럽고 맛있어요 건강한 밥상,"'음식이 맛있어요', '재료가 신선해요', '친절해요', '주차하기 편해요'",5월 29일 수요일,수요일,1
3,SunnyDay15,크루아상148 위례점,베이커리,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,아메리카노 2500원 무난한 맛이에요 아몬드 크로아상은 조금 드라이하게 느껴졌어요,"'가성비가 좋아요', '친절해요'",5월 28일 화요일,화요일,1
4,SunnyDay15,미도인 위례,양식,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,큐브덮밥 미도인 가케소바 무난 새우커틀렛은 다진 새우를 크로켓 모양으로 튀겼는데 좀...,"'인테리어가 멋져요', '주차하기 편해요'",5월 28일 화요일,화요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,samestage,LTP 한남,브런치,서울특별시 용산구 한남동,예약 후,바로 입장,데이트,아이,평일인데도 웨이팅 있는 핫플이네요\n시그니처브런치 비주얼도이쁘고 맛도 좋아요 \n사...,"'음식이 맛있어요', '재료가 신선해요', '메뉴 구성이 알차요', '인테리어가 멋...",1월 26일 금요일,금요일,1
1285,samestage,메이베이크,"카페,디저트",서울특별시 성동구 하왕십리동,예약 없이,바로 입장,일상,혼자,케이크 맛있는 상왕십리카페 메이베이크 딸기케이크 홀딱 반했어요 미니케이크라 3인가족...,"'디저트가 맛있어요', '빵이 맛있어요', '아늑해요', '친절해요'",1월 25일 목요일,목요일,1
1286,samestage,김치옥 삼송점,"육류,고기요리",경기도 고양시 덕양구 삼송동,예약 없이,바로 입장,데이트,연인·배우자,김치 전문점이라 그런가 김치찜이 엄지척입니다 제육볶음이랑 세뜨로 먹었어요 고기도 찐...,"'음식이 맛있어요', '반찬이 잘 나와요', '메뉴 구성이 알차요', '친절해요'",1월 24일 수요일,수요일,2
1287,samestage,AVEK CHERI,"카페,디저트",서울특별시 용산구 한남동,예약 없이,바로 입장,일상,혼자,소금빵맛집 잘찾아온거같아요 소프트소금빵 오리지널소금빵 둘다먹었는데 각자의매력이 있어...,"'커피가 맛있어요', '빵이 맛있어요', '인테리어가 멋져요', '뷰가 좋아요', ...",1월 22일 월요일,월요일,1


In [56]:
# 리뷰의 길이가 짧은 경우 제외시킴
new_df = new_df[new_df['place_review'].apply(lambda x: isinstance(x, str) and len(x.split()) > 1)]
new_df

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count
0,SunnyDay15,장안면옥 명일동점,냉면,서울특별시 강동구 명일동,정보없음,바로 입장,데이트,부모님,갈비탕 맛있어요,'음식이 맛있어요',5월 30일 목요일,목요일,5
1,SunnyDay15,스트라이프커피,카페,서울특별시 강남구 청담동,예약 없이,바로 입장,일상,혼자,동네 커피 맛집 원두가 고소하면서도 묵직해서 원두 \n사와서 집에서 드립으로 내려마...,"'커피가 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 29일 수요일,수요일,4
2,SunnyDay15,홍영재장수청국장,한정식,서울특별시 강남구 대치동,예약 없이,바로 입장,일상,연인·배우자,갈비찜 정식 먹었는데 부드럽고 맛있어요 건강한 밥상,"'음식이 맛있어요', '재료가 신선해요', '친절해요', '주차하기 편해요'",5월 29일 수요일,수요일,1
3,SunnyDay15,크루아상148 위례점,베이커리,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,아메리카노 2500원 무난한 맛이에요 아몬드 크로아상은 조금 드라이하게 느껴졌어요,"'가성비가 좋아요', '친절해요'",5월 28일 화요일,화요일,1
4,SunnyDay15,미도인 위례,양식,서울특별시 송파구 장지동,예약 없이,바로 입장,일상,친척·형제,큐브덮밥 미도인 가케소바 무난 새우커틀렛은 다진 새우를 크로켓 모양으로 튀겼는데 좀...,"'인테리어가 멋져요', '주차하기 편해요'",5월 28일 화요일,화요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,samestage,LTP 한남,브런치,서울특별시 용산구 한남동,예약 후,바로 입장,데이트,아이,평일인데도 웨이팅 있는 핫플이네요\n시그니처브런치 비주얼도이쁘고 맛도 좋아요 \n사...,"'음식이 맛있어요', '재료가 신선해요', '메뉴 구성이 알차요', '인테리어가 멋...",1월 26일 금요일,금요일,1
1285,samestage,메이베이크,"카페,디저트",서울특별시 성동구 하왕십리동,예약 없이,바로 입장,일상,혼자,케이크 맛있는 상왕십리카페 메이베이크 딸기케이크 홀딱 반했어요 미니케이크라 3인가족...,"'디저트가 맛있어요', '빵이 맛있어요', '아늑해요', '친절해요'",1월 25일 목요일,목요일,1
1286,samestage,김치옥 삼송점,"육류,고기요리",경기도 고양시 덕양구 삼송동,예약 없이,바로 입장,데이트,연인·배우자,김치 전문점이라 그런가 김치찜이 엄지척입니다 제육볶음이랑 세뜨로 먹었어요 고기도 찐...,"'음식이 맛있어요', '반찬이 잘 나와요', '메뉴 구성이 알차요', '친절해요'",1월 24일 수요일,수요일,2
1287,samestage,AVEK CHERI,"카페,디저트",서울특별시 용산구 한남동,예약 없이,바로 입장,일상,혼자,소금빵맛집 잘찾아온거같아요 소프트소금빵 오리지널소금빵 둘다먹었는데 각자의매력이 있어...,"'커피가 맛있어요', '빵이 맛있어요', '인테리어가 멋져요', '뷰가 좋아요', ...",1월 22일 월요일,월요일,1


> 1289 rows 에서 655 rows로

In [58]:
# 필요한 칼럼만 들고오기
data = new_df.filter(['user_name','place_review',	'place_tag', 'visit_count'])
data = data.reset_index()
data['index'] = data['index']+1
data

,index,user_name,place_review,place_tag,visit_count
0,1,SunnyDay15,갈비탕 맛있어요,'음식이 맛있어요',5
1,2,SunnyDay15,동네 커피 맛집 원두가 고소하면서도 묵직해서 원두 \n사와서 집에서 드립으로 내려마...,"'커피가 맛있어요', '+4\n개의 리뷰 더 있습니다'",4
2,3,SunnyDay15,갈비찜 정식 먹었는데 부드럽고 맛있어요 건강한 밥상,"'음식이 맛있어요', '재료가 신선해요', '친절해요', '주차하기 편해요'",1
3,4,SunnyDay15,아메리카노 2500원 무난한 맛이에요 아몬드 크로아상은 조금 드라이하게 느껴졌어요,"'가성비가 좋아요', '친절해요'",1
4,5,SunnyDay15,큐브덮밥 미도인 가케소바 무난 새우커틀렛은 다진 새우를 크로켓 모양으로 튀겼는데 좀...,"'인테리어가 멋져요', '주차하기 편해요'",1
...,...,...,...,...,...
650,1285,samestage,평일인데도 웨이팅 있는 핫플이네요\n시그니처브런치 비주얼도이쁘고 맛도 좋아요 \n사...,"'음식이 맛있어요', '재료가 신선해요', '메뉴 구성이 알차요', '인테리어가 멋...",1
651,1286,samestage,케이크 맛있는 상왕십리카페 메이베이크 딸기케이크 홀딱 반했어요 미니케이크라 3인가족...,"'디저트가 맛있어요', '빵이 맛있어요', '아늑해요', '친절해요'",1
652,1287,samestage,김치 전문점이라 그런가 김치찜이 엄지척입니다 제육볶음이랑 세뜨로 먹었어요 고기도 찐...,"'음식이 맛있어요', '반찬이 잘 나와요', '메뉴 구성이 알차요', '친절해요'",2
653,1288,samestage,소금빵맛집 잘찾아온거같아요 소프트소금빵 오리지널소금빵 둘다먹었는데 각자의매력이 있어...,"'커피가 맛있어요', '빵이 맛있어요', '인테리어가 멋져요', '뷰가 좋아요', ...",1


In [124]:
# 할당량 제한이 있으니까 조금만 테스트 (각 유저 별로 1개씩만 뽑기)
def sample(group):
  if len(group) >=2:
    return group.sample(1, replace=False)
  return group

# apply 함수를 사용하여 각 그룹에서 무작위 샘플링
sample_df = data.groupby('user_name').apply(sample).reset_index(drop=True)
sample_df

,index,user_name,place_review,place_tag,visit_count
0,452,5430583,모임장소로 좋아요,'단체모임 하기 좋아요',1
1,1006,O3O hana,방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평...,"'디저트가 맛있어요', '대화하기 좋아요', '아늑해요'",1
2,1185,OpotatoO,요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고 같이간 많이가는 단골소녀가 애...,"'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 ...",1
3,37,SunnyDay15,한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 ...,"'커피가 맛있어요', '대화하기 좋아요', '친절해요', '화장실이 깨끗해요'",1
4,1233,samestage,용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다,"'커피가 맛있어요', '인테리어가 멋져요', '사진이 잘 나와요', '아늑해요', ...",1
5,257,stay real,친절하고 맛있음\n주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요', '특별한 메뉴가 있어요...",28
6,331,스트로베리홍시,맛있어요옹 ㅜㅜㅜㅜ,"'음식이 맛있어요', '양이 많아요'",1
7,606,외계인의별,출장 중 라면 땡겨서 검색했다가\n1인 혼밥 가능한 항아리문어라면 있어서 먹으러왔어...,"'음식이 맛있어요', '재료가 신선해요', '특별한 메뉴가 있어요', '양이 많아요...",2
8,809,울랄라언니,계란장과 포케 조합 끝내주네요,"'음식이 맛있어요', '재료가 신선해요'",1


### *3. DataLoader*

In [125]:
# 데이터로드를 위해 loader 객체 생성
loader = DataFrameLoader(sample_df, page_content_column="index")
docs = loader.load()
docs

[Document(page_content='452', metadata={'user_name': '5430583', 'place_review': '모임장소로 좋아요', 'place_tag': "'단체모임 하기 좋아요'", 'visit_count': 1}),
 Document(page_content='1006', metadata={'user_name': 'O3O hana', 'place_review': '방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평소 달달한 디저트를 좋아해서 아이스크림도 종종 먹는데 코지 젤라또에서 4가지맛을 먹어봤는데 다 맛있더라고요\n위에 귀엽게 작은 미니콘과 다른맛도 하나 올라가서 더 좋아요\n방이먹자골목에서 디저트 아이스크림 땡기면 다시 방문하고 싶은곳이에요', 'place_tag': "'디저트가 맛있어요', '대화하기 좋아요', '아늑해요'", 'visit_count': 1}),
 Document(page_content='1185', metadata={'user_name': 'OpotatoO', 'place_review': '요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고  같이간 많이가는 단골소녀가 애견동반 은 전화 문의후 방문주세요 반려동물은 전화로 문의후 소형견가방과함께있으면 가능하다고 하셔서  데리고갔습니다 요거트가 넘 맛있어서 딸기우유랑 무가당요거트 사왔어요  무가당은 강\n내용 더보기', 'place_tag': "'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요', '메뉴 구성이 알차요'", 'visit_count': 1}),
 Document(page_content='37', metadata={'user_name': 'SunnyDay15', 'place_review': '한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 묵직하면서 좋더라구요 작은 야외 좌석도 있어요', 'place_tag': "'커피가 맛있어요',

### 4. TextSplitter

In [126]:
# 토큰제한으로 문자 split해야함.
character_text_spliter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits_docs = character_text_spliter.split_documents(docs)
splits_docs

[Document(page_content='452', metadata={'user_name': '5430583', 'place_review': '모임장소로 좋아요', 'place_tag': "'단체모임 하기 좋아요'", 'visit_count': 1}),
 Document(page_content='1006', metadata={'user_name': 'O3O hana', 'place_review': '방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평소 달달한 디저트를 좋아해서 아이스크림도 종종 먹는데 코지 젤라또에서 4가지맛을 먹어봤는데 다 맛있더라고요\n위에 귀엽게 작은 미니콘과 다른맛도 하나 올라가서 더 좋아요\n방이먹자골목에서 디저트 아이스크림 땡기면 다시 방문하고 싶은곳이에요', 'place_tag': "'디저트가 맛있어요', '대화하기 좋아요', '아늑해요'", 'visit_count': 1}),
 Document(page_content='1185', metadata={'user_name': 'OpotatoO', 'place_review': '요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고  같이간 많이가는 단골소녀가 애견동반 은 전화 문의후 방문주세요 반려동물은 전화로 문의후 소형견가방과함께있으면 가능하다고 하셔서  데리고갔습니다 요거트가 넘 맛있어서 딸기우유랑 무가당요거트 사왔어요  무가당은 강\n내용 더보기', 'place_tag': "'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요', '메뉴 구성이 알차요'", 'visit_count': 1}),
 Document(page_content='37', metadata={'user_name': 'SunnyDay15', 'place_review': '한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 묵직하면서 좋더라구요 작은 야외 좌석도 있어요', 'place_tag': "'커피가 맛있어요',

### 5. Embedding, Vectorstore

In [127]:
# 임베딩 후 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=splits_docs, embedding=HuggingFaceBgeEmbeddings())

# 벡터스토어 내 벡터들 간 유사도 측정
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 6. Prompt

In [128]:
prompt = ''' Classify the text into negative or positive.
review : {place_review}
classification : {classification}

place_review를 읽고 사용자의 리뷰가 긍정 혹은 부정인지 평가해줘.
만약 긍정이라면 1로 라벨링을 하고, 부정이라면 0으로 라벨링해줘.

- 긍정의 예시를 보여줄게.
'콩나물국밥 시원해요 김치도 맛있네요'는 긍정이야.
대체로 '맛있다', '또 가고 싶다' 처럼 긍정적인 문맥이면 긍정으로 평가해줘.
또, visit_count(재방문 횟수)가 많을수록 긍정으로 평가해줘.

- 부정의 예시를 보여줄게.
'아이스 아메리카노 너무 묽어서 재구매는 안 할꺼에요'는 부정이야.
대체로 '실망이다', '맛없다', '다시는 안갈듯', '재방문은 글쎄'처럼 부정적인 문맥이면 부정으로 평가해줘.'''

prompt_template = ChatPromptTemplate.from_template(prompt)

### 7. Model 생성

In [129]:
agent_gpt = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0.0, verbose=False, openai_api_key = os.environ['OPEN_API_KEY'])

lagchain = (
    {'classification':retriever, 'place_review':RunnablePassthrough()}
    | prompt_template #프롬프트
    | agent_gpt
    | StrOutputParser()
)

### 8. 긍정 부정 데이터 생성

In [130]:
classification_list = []

for document in docs:
  item_dict = {}
  item_dict['index'] = document.page_content

  for key, value in document.metadata.items():
    item_dict[key] = value

  # 긍정부정
  item_dict['classification'] = lagchain.invoke(item_dict['place_review'])
  classification_list.append(item_dict)


df_prompt_data = pd.DataFrame(classification_list)
df_prompt_data

,index,user_name,place_review,place_tag,visit_count,classification
0,452,5430583,모임장소로 좋아요,'단체모임 하기 좋아요',1,"[Document(page_content='1185', metadata={'user..."
1,1006,O3O hana,방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평...,"'디저트가 맛있어요', '대화하기 좋아요', '아늑해요'",1,1
2,1185,OpotatoO,요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고 같이간 많이가는 단골소녀가 애...,"'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 ...",1,1
3,37,SunnyDay15,한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 ...,"'커피가 맛있어요', '대화하기 좋아요', '친절해요', '화장실이 깨끗해요'",1,"Document(page_content='606', metadata={'user_n..."
4,1233,samestage,용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다,"'커피가 맛있어요', '인테리어가 멋져요', '사진이 잘 나와요', '아늑해요', ...",1,"긍정적인 리뷰 예시:\n- ""분위기도 좋고 커피 찐맛입니다""\n- ""맛있어요옹""\n..."
5,257,stay real,친절하고 맛있음\n주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요', '특별한 메뉴가 있어요...",28,"리뷰 ""친절하고 맛있음 주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함""은 긍..."
6,331,스트로베리홍시,맛있어요옹 ㅜㅜㅜㅜ,"'음식이 맛있어요', '양이 많아요'",1,"리뷰 ""맛있어요옹 ㅜㅜㅜㅜ""는 긍정적인 문맥이므로 1로 라벨링합니다."
7,606,외계인의별,출장 중 라면 땡겨서 검색했다가\n1인 혼밥 가능한 항아리문어라면 있어서 먹으러왔어...,"'음식이 맛있어요', '재료가 신선해요', '특별한 메뉴가 있어요', '양이 많아요...",2,부정
8,809,울랄라언니,계란장과 포케 조합 끝내주네요,"'음식이 맛있어요', '재료가 신선해요'",1,"Document(page_content='331', metadata={'user_n..."


> 프롬프트를 잘못 작성한 것 같다. <br>
> 프롬프트에 추가할 것 : 'classification 에는 1,0만 써줘', '이유는 쓰지마'

In [140]:
for i in df_prompt_data['classification']:
  print(i)

[Document(page_content='1185', metadata={'user_name': 'OpotatoO', 'place_review': '요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고  같이간 많이가는 단골소녀가 애견동반 은 전화 문의후 방문주세요 반려동물은 전화로 문의후 소형견가방과함께있으면 가능하다고 하셔서  데리고갔습니다 요거트가 넘 맛있어서 딸기우유랑 무가당요거트 사왔어요  무가당은 강\n내용 더보기', 'place_tag': "'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요', '메뉴 구성이 알차요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='1233', metadata={'user_name': 'samestage', 'place_review': '용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다', 'place_tag': "'커피가 맛있어요', '인테리어가 멋져요', '사진이 잘 나와요', '아늑해요', '친절해요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='331', metadata={'user_name': '스트로베리홍시', 'place_review': '맛있어요옹 ㅜㅜㅜㅜ', 'place_tag': "'음식이 맛있어요', '양이 많아요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='809', metadata={'user_name': '울랄라언니', 'place_review': '계란장과 포케 조합 끝내주네요', 'place_tag': "'음식이 맛있어요', '재료가 신선해요'", 'visit_count': 1})] -> Positive (1)
1
1
Document(page_content='606', metadata={'user_name': '외계인의별', 

> 결과 요약 <br>
> Document(page_content='1185'): 1 <br>
> Document(page_content='1233') : 1 <br>
> Document(page_content='331') : 1 <br>
> Document(page_content='809') : 1 <br>
> Document(page_content='606') : 1 <br>
> Document(page_content='331) :1 <br>
> Document(page_content='452') : 1 <br>
> Document(page_content='37') : 1 <br>
> Document(page_content='606') : 1 <br>


In [141]:
df_prompt_data['labeling'] = 1

In [142]:
df_prompt_data[['place_review','classification', 'labeling']]

,place_review,classification,labeling
0,모임장소로 좋아요,"[Document(page_content='1185', metadata={'user...",1
1,방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평...,1,1
2,요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고 같이간 많이가는 단골소녀가 애...,1,1
3,한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 ...,"Document(page_content='606', metadata={'user_n...",1
4,용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다,"긍정적인 리뷰 예시:\n- ""분위기도 좋고 커피 찐맛입니다""\n- ""맛있어요옹""\n...",1
5,친절하고 맛있음\n주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함,"리뷰 ""친절하고 맛있음 주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함""은 긍...",1
6,맛있어요옹 ㅜㅜㅜㅜ,"리뷰 ""맛있어요옹 ㅜㅜㅜㅜ""는 긍정적인 문맥이므로 1로 라벨링합니다.",1
7,출장 중 라면 땡겨서 검색했다가\n1인 혼밥 가능한 항아리문어라면 있어서 먹으러왔어...,부정,1
8,계란장과 포케 조합 끝내주네요,"Document(page_content='331', metadata={'user_n...",1


> 문제 : 부정적인 리뷰 수가 적다.

****************